In [1]:
from yugiquery import *
header('Rush Duel')

<div align='center'>
    <br>
    <!-- Pre Style needed to fix HTML formatting -->
    <pre style="line-height: var(--jp-code-line-height); font-family: var(--jp-code-font-family)">
    ██    ██ ██    ██  ██████  ██  ██████  ██    ██ ███████ ██████  ██    ██ 
     ██  ██  ██    ██ ██       ██ ██    ██ ██    ██ ██      ██   ██  ██  ██  
      ████   ██    ██ ██   ███ ██ ██    ██ ██    ██ █████   ██████    ████   
       ██    ██    ██ ██    ██ ██ ██ ▄▄ ██ ██    ██ ██      ██   ██    ██    
       ██     ██████   ██████  ██  ██████   ██████  ███████ ██   ██    ██    
                                      ▀▀                                     
    </pre>
</div>
<div align='right'>
    Rush Duel - Execution started 12/02/2023 00:07 UTC <br>
    By <b>Guilherme Ruiz</b>
</div>

---

In [2]:
rush_df = fetch_rush()

1770 results



In [3]:
rush_df

,Name,Password,Card type,Property,Primary type,Secondary type,Attribute,Monster type,Level/Rank,ATK,DEF,Pendulum Scale,Link,Link Arrows,Effect type,Archseries,TCG status,OCG status,Artwork,Page name,Page URL,Modification date
0,1-Up,NaN,Trap Card,Normal Trap Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Effect,)",NaN,NaN,NaN,NaN,1-Up,https://yugipedia.com/wiki/1-Up,2022-09-18 19:47:54
1,1/Infinity,NaN,Spell Card,Normal Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/Infinity,https://yugipedia.com/wiki/1/Infinity,2021-12-19 01:15:37
2,1seg Wurm,NaN,Monster Card,NaN,Effect Monster,NaN,LIGHT,Dragon,1,700,1100,NaN,NaN,NaN,NaN,"(High Tech Dragon,)",NaN,NaN,NaN,1seg Wurm,https://yugipedia.com/wiki/1seg_Wurm,2023-01-22 00:40:17
3,300 Light-Year Red Cloak,NaN,Spell Card,Normal Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Effect,)","(Cosmic Urban Legend,)",NaN,NaN,NaN,300 Light-Year Red Cloak,https://yugipedia.com/wiki/300_Light-Year_Red_...,2022-11-17 12:07:53
4,7 Chance,NaN,Trap Card,Normal Trap Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Effect,)",NaN,NaN,NaN,NaN,7 Chance,https://yugipedia.com/wiki/7_Chance,2022-12-10 20:50:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,Zero Drake,NaN,Monster Card,NaN,Normal Monster,NaN,WATER,Dragon,1,0,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zero Drake,https://yugipedia.com/wiki/Zero_Drake,2023-02-05 22:13:12
1766,Zero-Sum Game,NaN,Spell Card,Normal Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zero-Sum Game,https://yugipedia.com/wiki/Zero-Sum_Game,2023-02-03 00:56:44
1767,Zombie Carnival,NaN,Spell Card,Field Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Effect,)",NaN,NaN,NaN,NaN,Zombie Carnival,https://yugipedia.com/wiki/Zombie_Carnival,2023-01-05 23:00:01
1768,Zombie Fireworks,NaN,Spell Card,Normal Spell Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Effect,)",NaN,NaN,NaN,NaN,Zombie Fireworks,https://yugipedia.com/wiki/Zombie_Fireworks,2022-10-20 06:57:57


In [4]:
footer()

<div align='right'>
    Execution completed 12/02/2023 00:08 UTC
</div>

## HTML export

In [5]:
# Save notebook on disck before generating HTML report
save_notebook()

Notebook saved to disk


In [6]:
! jupyter nbconvert Rush.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

[NbConvertApp] Converting notebook Rush.ipynb to HTML


[NbConvertApp] Writing 585018 bytes to ../Rush.html


## Git

In [ ]:
! git add "../*[Rr]ush*"

In [ ]:
! git commit -m {"'Rush duel update-" + timestamp.isoformat() + "'"}